In [256]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

In [257]:
# Importamos Librerias
import numpy as np
import math
from collections import Counter
from sortedcontainers import SortedDict
import sklearn.naive_bayes as nb
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict

In [258]:
class Datos:

  TiposDeAtributos=('Continuo','Nominal')

  # TODO: procesar el fichero para asignar correctamente las variables tipoAtributos, nombreAtributos, nominalAtributos, datos y diccionarios
  # NOTA: No confundir TiposDeAtributos con tipoAtributos
  def __init__(self, nombreFichero):

      with open(nombreFichero, "r") as f:
        # Guardamos el numero de datos que contiene el DataSet y esta en la primera linea
        self.numDatos = int(f.readline())

        # Guardamos el nombre de los atributos
        self.nombreAtributos = f.readline().strip('\n').split(',')
        #print(self.nombreAtributos)

        # Leemos el tipo de los atributos de las variables y eliminamos el ultimo que es un salto de linea
        self.tipoAtributos = f.readline().strip('\n').split(',')
        #print(self.tipoAtributos)

        # Comprobamos que todos los atributos sean Continuos o Nominales
        if any(atr not in Datos.TiposDeAtributos for atr in self.tipoAtributos):
            raise ValueError("Tipo de atributo erroneo")

        # Segun el atributo, asignamos True o False.
        self.nominalAtributos = []

        # Guardamos en la lista nominalAtributos en la posicion de cada uno si es o no Nominal
        for tipo in self.tipoAtributos:
            if tipo == self.TiposDeAtributos[0]:
                self.nominalAtributos.append(False)
            else:
                self.nominalAtributos.append(True)
        #print(self.nominalAtributos)

        # Guardamos los datos del fichero y los formateamos, de tal forma que cada linea es una lista
        datos = f.readlines()
        datosFormat = []
        for lista in datos:
            datosFormat.append(lista.strip('\n').split(','))

        # print(set(sorted(datosFormat[0])))
        listaDatosAtributos = []
        for i in range(len(self.tipoAtributos)):
            listaDatosAtributos.append([])

        # Hacemos la traspuesta de los datos que guardamos para que cada lista de atributo guarde todos los datos
        # de cada atributo.
        for lista in datosFormat:
            i = 0
            for item in lista:
                listaDatosAtributos[i].append(item)
                i += 1

        # Ordenamos y hacemos un set para eliminar repetidos.
        i = 0
        for item in listaDatosAtributos:
            listaDatosAtributos[i] = sorted(set(item))
            i += 1


        # Creacion de lista diccionarios, en caso de que el atributo sea Continuo, el diccionario estara vacio
        self.listaDicts = []
        for i in range(len(self.tipoAtributos)):
            self.listaDicts.append({})

        # Creamos el diccionario tal y como se describe en las diapositivas, por orden y asignando valores numericos crecientes
        i = 0
        for atributo in listaDatosAtributos:
            k = 0
            if self.tipoAtributos[i] == "Nominal":
                for dato in atributo:
                    self.listaDicts[i][dato] = k
                    k += 1
            i += 1

        # Creacion de la matriz de datos utilizando el diccionario para mapear los valores
        # En primer lugar, creamos una matriz vacia de tamaña numero de atributos.
        self.datos = np.empty((int(self.numDatos),int(len(self.tipoAtributos))))
        i = 0
        j = 0

        # Metemos los datos en la matriz, mapeando con los diccionarios en el caso de que sean Nominales, y si son continuos normal.
        for i in range(int(self.numDatos)):
            for j in range(len(self.tipoAtributos)):
                if self.tipoAtributos[j] == 'Nominal':
                    self.datos[i][j] = self.listaDicts[j].get(str(datosFormat[i][j]))
                else:
                    self.datos[i][j] = datosFormat[i][j]
        
        print(self.nombreAtributos)
        print(self.listaDicts)
        f.close()


  # TODO: implementar en la practica 1
  def extraeDatos(self, idx):
    return self.datos[idx]


In [259]:
dataset = Datos('lenses.data')

print(dataset.datos)

['Age', 'Spectacle', 'Astigmatic', 'Tear', 'Class']
[{'1': 0, '2': 1, '3': 2}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1, '3': 2}]
[[0. 0. 0. 0. 2.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 2.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 2.]
 [0. 1. 0. 1. 1.]
 [0. 1. 1. 0. 2.]
 [0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 2.]
 [1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 2.]
 [1. 0. 1. 1. 0.]
 [1. 1. 0. 0. 2.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 2.]
 [1. 1. 1. 1. 2.]
 [2. 0. 0. 0. 2.]
 [2. 0. 0. 1. 2.]
 [2. 0. 1. 0. 2.]
 [2. 0. 1. 1. 0.]
 [2. 1. 0. 0. 2.]
 [2. 1. 0. 1. 1.]
 [2. 1. 1. 0. 2.]
 [2. 1. 1. 1. 2.]]


In [260]:
from abc import ABCMeta,abstractmethod


class Particion():

  # Esta clase mantiene la lista de �ndices de Train y Test para cada partici�n del conjunto de particiones
    def __init__(self,train=[],test=[]):
        self.indicesTrain=train
        self.indicesTest=test
    
    def __str__(self):
        return "Train: {}\nTest:  {}".format(str(self.indicesTrain),str(self.indicesTest)) 

#####################################################################################################

class EstrategiaParticionado:

    # Clase abstracta
    __metaclass__ = ABCMeta

    # Lista de las particiones
    def __init__(self, nombre=""):
        self.nombreEstrategia = nombre
        self.numeroParticiones = 0
        self.particiones=[]

    # Atributos: deben rellenarse adecuadamente para cada estrategia concreta: nombreEstrategia, numeroParticiones, listaParticiones. Se pasan en el constructor

    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada estrategia concreta
    def creaParticiones(self,datos,seed=None):
        pass

In [261]:
class ValidacionSimple(EstrategiaParticionado):
    
    def __init__(self, porcentaje):
        self.porcentaje = porcentaje
        super().__init__("Validacion simple")
        
  # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado.
  # Devuelve una lista de particiones (clase Particion)
  # TODO: implementar
    def creaParticiones(self,datos,seed=None):
        np.random.seed(seed)

        self.numeroParticiones = 1
    
        # Generamos una lista con todos los números de datos aleatorios   
        indicesAleatorios = np.random.permutation(int(datos.numDatos))
        
        # Creamos la particion, en funcion del porcentaje especificado
        self.particiones = [Particion(indicesAleatorios[:int(datos.numDatos*self.porcentaje)],
                                      indicesAleatorios[int(datos.numDatos*self.porcentaje):])]
        
        return self.particiones
    

In [262]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
print(validacion_simple.particiones[0])

Train: [ 5 16  1 20 17 19  4 18 15 21  2 13 22  3 10  6 12  9]
Test:  [ 8  7 11 14  0 23]


In [263]:
class ValidacionCruzada(EstrategiaParticionado):

  # Crea particiones segun el metodo de validacion cruzada.
  # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
  # Esta funcion devuelve una lista de particiones (clase Particion)
  # TODO: implementar
    
    def __init__(self, k):
        self.k = k
        super().__init__("Validacion cruzada")

    def creaParticiones(self,datos,seed=None):
        np.random.seed(seed)
        
        self.numeroParticiones = self.k
        
        # Generamos una lista con todos los números de datos aleatorios   
        indicesAleatorios = np.random.permutation(int(datos.numDatos))
        
        # Hallamos el tamaño de cada bloque
        tamBloque = int(datos.numDatos/self.k)
        
        
        datosSobran = datos.numDatos - (tamBloque*self.k)
        count = 0
        for i in range(self.k):
            
            train = np.delete(indicesAleatorios, range(i*tamBloque,(i+1)*tamBloque))
            test =  indicesAleatorios[i*tamBloque:(i+1)*tamBloque]
            
            # Caso en el que la cuenta es justa
            if datosSobran == 0:
                self.particiones.append(Particion(train, test))
                
            # Contemplamos el caso de que la division para sacar el numero de subconjuntos no fuese entera
            if datosSobran > 0:
                count += 1
                particionTest = np.append(test, train[(datos.numDatos - tamBloque)- i - 1])
                particionTrain = np.delete(train, (datos.numDatos - tamBloque)- i - 1)
                datosSobran -= 1
                self.particiones.append(Particion(particionTrain, particionTest))
                
            
           
                

In [264]:
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

for particion in v_cruzada.particiones:
    print(particion)

Train: [21  8 12 17 14 20 19 18 11 10  7  9 13  4  5 16 23  2]
Test:  [ 0  6  1 22 15  3]
Train: [ 0  6  1 22 15  3 19 18 11 10  7  9 13  4  5 16 23  2]
Test:  [21  8 12 17 14 20]
Train: [ 0  6  1 22 15  3 21  8 12 17 14 20 13  4  5 16 23  2]
Test:  [19 18 11 10  7  9]
Train: [ 0  6  1 22 15  3 21  8 12 17 14 20 19 18 11 10  7  9]
Test:  [13  4  5 16 23  2]


In [265]:
class Clasificador:
  
    # Clase abstracta
    __metaclass__ = ABCMeta

    # Metodos abstractos que se implementan en casa clasificador concreto
    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # datosTrain: matriz numpy con los datos de entrenamiento
    # atributosDiscretos: array bool con la indicatriz de los atributos nominales
    # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion de variables discretas
    def entrenamiento(self,datos,datosTrain,atributosDiscretos,diccionario):
        pass


    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # devuelve un numpy array con las predicciones
    def clasifica(self,datosTest,atributosDiscretos,diccionario):
        pass


    # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
    # TODO: implementar
    def error(self,datos,pred):
    # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error    
        i = 0
        real = datos[:,-1]
        error = 0
        for i in range(len(real)):
            if real[i] != pred[i]:
                error += 1
        err = (error)/(len(real)+0.0)
        return err


    # Realiza una clasificacion utilizando una estrategia de particionado determinada
    # TODO: implementar esta funcion
    def validacion(self,particionado,dataset,clasificador,seed=None):

    # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
    # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
    # y obtenemos el error en la particion de test i
    # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
    # y obtenemos el error en la particion test. Otra opci�n es repetir la validaci�n simple un n�mero especificado de veces, obteniendo en cada una un error. Finalmente se calcular�a la media.
        errores = 0
        #particionado.creaParticiones(dataset, seed)
    # Comprobamos si es por validación cruzada o simple, por la longitud de la lista de particiones
    # Validación Simple
        if len(particionado.particiones) == 1:
            clasificador.entrenamiento(dataset, particionado.particiones[0].indicesTrain)
            pred = clasificador.clasifica(dataset,particionado.particiones[0].indicesTest)
            ret = self.error(dataset.extraeDatos(particionado.particiones[0].indicesTest), pred)
            if ret > 0:
                return ret
            else:
                return 0
            
    # Validación Cruzada        
        else:
            for particion in particionado.particiones:
                clasificador.entrenamiento(dataset, particion.indicesTrain)
                pred = clasificador.clasifica(dataset,particion.indicesTest)
                ret = self.error(dataset.extraeDatos(particion.indicesTest), pred)
                errores += ret
            error = errores/len(particionado.particiones)
            #Devolucion de la media de los errores
            return error

In [274]:
class ClasificadorNaiveBayes(Clasificador):
    
    def __init__(self, laplace):
        self.laplace = laplace
        
    
    def entrenamiento(self,dataset,datosTrain):
     
        # Cargamos todos los datos de la clase del dataset desde la matriz de datos
        clasesTrain = dataset.extraeDatos(datosTrain)
        self.numClases = clasesTrain[:,-1]
        # Contamos las apariciones de cada uno para luego calcular la probabilidad a priori de cada clase
        counter = Counter(self.numClases)
        # Calculamos la probabilidad de la clase y lo metemos en un diccionario ordenado segun el numero 
        # correspondiente a cada clase asignado en el diccionario
        self.dictPrioris={}
        for k in counter:
            k = int(k)
            counter[k] = counter[k]/len(self.numClases)
            self.dictPrioris[k] = counter[k]
            
        # Aqui ordenamos el diccionario para que esten en el mismo orden de como extraemos los datos del dataset
        self.dictPrioris = SortedDict(self.dictPrioris)
        
        # Calcular tablas de probabilidades del entrenamiento. Tenemos que calcular por cada atributo una cuenta
        # de las apariciones en cada clase
        # Creamos una lista de matrices, donde vamos almacenar todos los datos que hemos obtenido en los datos de Test
        self.posteriori = np.zeros(len(dataset.nombreAtributos)-1,dtype=object)
        
        # Recorremos todos los datos de la matriz sin llegar a la clase
        for i in range(len(dataset.nombreAtributos) - 1):
            
            # Si el dato que obtenemos es Nominal haremos el recuento de todas las veces que sale la P(D|H)
            if dataset.nominalAtributos[i] == True:
                
                # Creamos una matriz de tamaño X: Número de Atributos menos la clase Y: Número de clases
                post = np.zeros((len(dataset.listaDicts[i]),len(dataset.listaDicts[-1])))
               
                # Aqui contamos todos las datos que queremos del datos Train para construir la matriz de entrenamiento
                for c in range(len(dataset.listaDicts[-1])):
                    datosEnt = dataset.extraeDatos(datosTrain)
                    dat = datosEnt[:,i]
                    repes = Counter(dat[datosEnt[:,-1] == c])
                    for r in repes:
                        post[int(r),c] = repes[r]
                    if self.laplace == True:
                        self.posteriori[i] = post +1
                    else:
                        self.posteriori[i] = post
            
            # Si el dato es Continuo obtendremos la media y la desviación tipica de la clase
            else:
                
                # Creamos una matriz de X: Los datos de Media y Desivación típica Y: Número de clases
                post = np.zeros((2,len(dataset.listaDicts[-1])))
               
                # Aqui obtenemos la media y desviación tipica de cada clase, despues de tener los datos de entrenamiento
                for c in range(len(dataset.listaDicts[-1])):
                    datosEnt = dataset.extraeDatos(datosTrain)
                    dat = datosEnt[:,i]
                    datos = dat[datosEnt[:,-1] == c]
                    post[0][c] = np.mean(datos)
                    post[1][c] = np.std(datos)
                self.posteriori[i] = post
            
        #Calculamos los valores de los posteriori de todos las tablas anteriores
        for j in range(len(dataset.nominalAtributos)-1):
            if dataset.nominalAtributos[j] == True:
                for i in range(len(dataset.listaDicts) - 1):
                    self.posteriori[i] /= sum(self.posteriori[i])

        
    def clasifica(self,dataset,datosTest):
        j = 0
        aux = 1
        aux2 = 1
        self.prediccion = []
        datTest = dataset.extraeDatos(datosTest)
        #Ahora vamos a estudiar la probabilidad de la clase con los datos obtenidos en el entrenamiento
        #Recorremos todos las datos de la matriz de los datos Test
        
        for dato in datTest:
            mapa = []
            listaVerosimilitudes = []
            #Aqui obtenemos los prioris de cada clase para poder obtener la probabilidad de cada una
            for clase in range(len(self.dictPrioris)):
            #Aqui obtenemos cada valor posteriori de nuestro entrenamiento de los datos, es decir, P(D|H)
                for atributo in range(len(self.posteriori)):
                    if dataset.nominalAtributos[atributo] == True:
                        aux = self.posteriori[atributo][int(dato[atributo])][clase]
                        listaVerosimilitudes.append(aux)
                    #Aqui obtenemos la siguiente probabilidad P(D|H)*P(H)
                        #aux2 = self.dictPrioris.get(clase)*aux
                        #aux = 1
                
                    #Lo añadimos a una lista para obtener la probabilidad de las diferentes clases
                        #mapa.append(aux2)
            
            #Aqui obtenemos la probabilidad de los atibutos continuos
                    else:
                        # Hacemos la formula de la distribucion normal
                        exp1 = 1/(self.posteriori[atributo][1][clase]*math.sqrt(2*math.pi))
                        exp2 = np.power((dato[atributo]-self.posteriori[atributo][0][clase]) ,2)
                        exp3 = np.power(self.posteriori[atributo][1][clase],2)
                        exp4 = exp2/exp3   
                        exp4 = math.exp((-1/2)* exp3)
                        aux = exp1 * exp4
                        listaVerosimilitudes.append(aux)
                    #aux2 = aux * self.dictPrioris.get(clase)
                    for verosimilitud in listaVerosimilitudes:
                        aux2 *= verosimilitud
                    aux2 *= self.dictPrioris.get(clase)
                    mapa.append(aux2)
            
            #Aqui obtenemos la predicción de mayor probabilidad y la guardamos en nuestra lista de predicciones
            self.prediccion.append(np.argmax(mapa))
        
        #Devolvemos la lista con la predicción de nuestro clasifica   
        return self.prediccion

In [275]:
clasificador = ClasificadorNaiveBayes(True)
#clasificador.entrenamiento(dataset, validacion_simple.particiones[0].indicesTrain)
#clasificador.clasifica(dataset, validacion_simple.particiones[0].indicesTest)
error = clasificador.validacion(validacion_simple,dataset,clasificador)
error1 = clasificador.validacion(v_cruzada,dataset,clasificador)
print("El error del Clasificador NaiveBayes  para Validacion Simple es: "+str(error))
print("El error del Clasificador NaiveBayes para Validacion Cruzada es: "+str(error1))


TypeError: only size-1 arrays can be converted to Python scalars

In [276]:
dataset = Datos('tic-tac-toe.data')
print(dataset.datos)

['TLeftSq', 'TMidSq', 'TRightSq', 'MLeftSq', 'MMidSq', 'MRightSq', 'BLeftSq', 'BMidSq', 'BRightSq', 'Class']
[{'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'negative': 0, 'positive': 1}]
[[2. 2. 2. ... 1. 1. 1.]
 [2. 2. 2. ... 2. 1. 1.]
 [2. 2. 2. ... 1. 2. 1.]
 ...
 [1. 2. 1. ... 1. 2. 0.]
 [1. 2. 1. ... 1. 2. 0.]
 [1. 1. 2. ... 2. 2. 0.]]


In [277]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

print("VALIDACION SIMPLE",validacion_simple.particiones[0])

print("VALIDACION CRUZADA")
for particion in v_cruzada.particiones:
    print(particion)


VALIDACION SIMPLE Train: [ 69 840 850 718 541 916 106 910 306  35 229 546 374 778 903 885 185  33
 182  60 331 614 409 689 172 640 254 325 192 741 294 427 152  74 765 560
 186 658 589 313 272 410 240 446  19 109 393 609 715 119 533 833 940 300
 779 333 198 180  14 856 150 528 390 157 357 768  81 857 561 890 303  64
 535 190 794 881 836 644 709 642  68 935 316 672 162 284 554 909 188 607
 619 617 412 415 563 674 868 666 544 138 829 259 448 505 558 716 117 101
 870 711 520 746 273 766  41 334 690 657 481 722 945 866 739 341 625 457
 459  40 467 817 255 841  77 136 456 557 361 914 204 626 299 613 382 276
 736 641 864 576 228 612 693 469 618 745 295 555 388 203 453 591 869 943
 407 782 728   1 628 556 726 241 954 953 703 932 936 789 783 629 222 898
 320 568 441  20 487 403  13 336 861 624 887  42  83 307 113 494 719 748
 353 776 750 777 207 790 842 305 634 880 280 754 475 116 845 495  62 616
 534 706 838 174 443 144 137 860 518 925 428 478 740 392 134 149 749 735
 145 194 565 742 673 552 9

In [278]:
clasificador = ClasificadorNaiveBayes(True)
#clasificador.entrenamiento(dataset, validacion_simple.particiones[0].indicesTrain)
#clasificador.clasifica(dataset, validacion_simple.particiones[0].indicesTest)
error = clasificador.validacion(validacion_simple,dataset,clasificador)
error1 = clasificador.validacion(v_cruzada,dataset,clasificador)
print("El error del Clasificador NaiveBayes  para Validacion Simple es: "+str(error))
print("El error del Clasificador NaiveBayes para Validacion Cruzada es: "+str(error1))

El error del Clasificador NaiveBayes  para Validacion Simple es: 0.6583333333333333
El error del Clasificador NaiveBayes para Validacion Cruzada es: 0.6534257322175732


In [279]:
dataset = Datos('german.data')
print(dataset.datos)
print(dataset.nominalAtributos)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'Class']
[{'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}, {}, {'A30': 0, 'A31': 1, 'A32': 2, 'A33': 3, 'A34': 4}, {'A40': 0, 'A41': 1, 'A410': 2, 'A42': 3, 'A43': 4, 'A44': 5, 'A45': 6, 'A46': 7, 'A48': 8, 'A49': 9}, {}, {'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}, {'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}, {}, {'A91': 0, 'A92': 1, 'A93': 2, 'A94': 3}, {'A101': 0, 'A102': 1, 'A103': 2}, {}, {'A121': 0, 'A122': 1, 'A123': 2, 'A124': 3}, {}, {'A141': 0, 'A142': 1, 'A143': 2}, {'A151': 0, 'A152': 1, 'A153': 2}, {}, {'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3}, {}, {'A191': 0, 'A192': 1}, {'A201': 0, 'A202': 1}, {'1': 0, '2': 1}]
[[ 0.  6.  4. ...  1.  0.  0.]
 [ 1. 48.  2. ...  0.  0.  1.]
 [ 3. 12.  4. ...  0.  0.  0.]
 ...
 [ 3. 12.  2. ...  0.  0.  0.]
 [ 0. 45.  2. ...  1.  0.  1.]
 [ 1. 45.  4. ...  0.  0.  0.]]
[True, False, True, True, False,

In [280]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

print("VALIDACION SIMPLE",validacion_simple.particiones[0])

print("VALIDACION CRUZADA")
for particion in v_cruzada.particiones:
    print(particion)

VALIDACION SIMPLE Train: [286 363 976 946 215 648 356 632 936 113 248 384 106 383 709 958 952 483
  30 209  27 739 723 104 524 289  22 969 649 807 766 737  41 774 746 537
 831 430 697 987 624  95  23 315  16 761 550 718 525 518 922 710 139 675
 111 736 777 488 487 154 912 876 210  10 138  13 583  86 277 713 181 872
 120  56 473 916 466 674 461 750 653 480 192 695 174 651 378 862 848  89
  60 825 386 569  62 532 874 763 422 257 692 571 197 190 638  25 178 547
 307 741 457 475 541 942 219 860 711 443 837 141 539 792 527 453 933 368
  14 147 968 431 464 371 426 246 606 870 953 633 826 281 344 556 167 612
 793 887 749 622 913 463 852 978 997 745 585 511 879 621 691 716 896  53
 785 365 419 157 882  48 298 451 109 489 884 251 565 411 346 983 602 495
   5 406 748 714 944 735 352 999 468 706 222 374 514 963 108 868 904 221
 574 490 211 255 821  49 889 254  28 469 630 364 439 935 545 670 762  47
 280 682 437 316 666 149 742 449  75 859  81 809 854 689 820 305 819 398
 945 900 216 857 107 961 5

In [281]:
clasificador = ClasificadorNaiveBayes(True)
#clasificador.entrenamiento(dataset, validacion_simple.particiones[0].indicesTrain)
#clasificador.clasifica(dataset, validacion_simple.particiones[0].indicesTest)
error = clasificador.validacion(validacion_simple,dataset,clasificador)
error1 = clasificador.validacion(v_cruzada,dataset,clasificador)
print("El error del Clasificador NaiveBayes  para Validacion Simple es: "+str(error))
print("El error del Clasificador NaiveBayes para Validacion Cruzada es: "+str(error1))

TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
#SKLearn Validacion Cruzada
# Hacemos el encode de los datos
encAtributos = preprocessing.OneHotEncoder(categorical_features=dataset.nominalAtributos[:-1],sparse=False)
X = encAtributos.fit_transform(dataset.datos[:,:-1])
Y = dataset.datos[:,-1]

# Partimos los datos en Train y Test
x_train, x_test, y_train,y_test = train_test_split(X,Y,test_size=0.2)

# Aplicamos Naive Bayes sin Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple sin correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)

# Aplicamos Naive Bayes con Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple con correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)



In [ ]:
# SKLearn Validacion Cruzada
# Hacemos el encode de los datos
encAtributos = preprocessing.OneHotEncoder(categorical_features=dataset.nominalAtributos[:-1],sparse=False)
X = encAtributos.fit_transform(dataset.datos[:,:-1])
Y = dataset.datos[:,-1]

# Partimos los datos en Train y Test
x_train, x_test, y_train,y_test = train_test_split(X,Y,test_size=0.2)

# Aplicamos Naive Bayes sin Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple sin correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)

# Aplicamos Naive Bayes con Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple con correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)
